In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [2]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(2,a)
    return 2*8^ds

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=max(mlist) # probably bc max(mlist) = lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)
        oddpart=a/2^d
        #ord(2,tau(2^d))= (I believe) 2d
        # so ord(2,tau(purepart))=ord(2,tau(2^d)) = 2d
        # so tau(purepart) = 2^(2d)
        # so tau(a) = tau(purepart)*tau(oddpart)=
        # 4^d*tau(oddpart); then we replace tau(oddpart)
        # with sigma(oddpart,k=11) by Ramanujan's congruence
        term=4^d*sigma(oddpart,k=11)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1000)

no=[]
for b in [1..5]:
    print()
    ls=[]
    for a in [1..500]: # 100 < precision is necessary
        start=time.time()
        c=p2RecipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds+order(2,a+1) # 3rd term based on some observations
        finish=time.time()
        print((b,a,test,finish-start))
        if test!=0:ls=ls+[a]
    print()
    print("bad a's for trial ",b);print();print((ls))
    no=no+[(b,len(ls))]
print("------------------------------------------------------------------")
print("no:");print();print(no)


(1, 1, 0, 0.04134321212768555)
(1, 2, 0, 0.07362508773803711)
(1, 3, 0, 0.08583188056945801)
(1, 4, 0, 0.09041810035705566)
(1, 5, 0, 0.07281112670898438)
(1, 6, 0, 0.08403897285461426)
(1, 7, 0, 0.11819696426391602)
(1, 8, 0, 0.10554099082946777)
(1, 9, 0, 0.12588191032409668)
(1, 10, 0, 0.12535500526428223)
(1, 11, 0, 0.11990618705749512)
(1, 12, 0, 0.13873791694641113)
(1, 13, 0, 0.09925580024719238)
(1, 14, 0, 0.1287529468536377)
(1, 15, 0, 0.18254375457763672)
(1, 16, 0, 0.17494988441467285)
(1, 17, 0, 0.1628100872039795)
(1, 18, 0, 0.17426276206970215)
(1, 19, 0, 0.17339205741882324)
(1, 20, 0, 0.17661213874816895)
(1, 21, 0, 0.1332240104675293)
(1, 22, 0, 0.17305493354797363)
(1, 23, 0, 0.13481545448303223)
(1, 24, 0, 0.15110492706298828)
(1, 25, 0, 0.1512918472290039)
(1, 26, 0, 0.1719212532043457)
(1, 27, 0, 0.16986298561096191)
(1, 28, 0, 0.1715998649597168)
(1, 29, 0, 0.15652823448181152)
(1, 30, 0, 0.19467806816101074)
(1, 31, 0, 0.21453213691711426)
(1, 32, 0, 0.213560819

(1, 257, 0, 0.5293459892272949)
(1, 258, 0, 0.48188304901123047)
(1, 259, 0, 0.5523099899291992)
(1, 260, 0, 0.5279500484466553)
(1, 261, 0, 0.559363842010498)
(1, 262, 0, 0.5149698257446289)
(1, 263, 0, 0.5506718158721924)
(1, 264, 0, 0.4724578857421875)
(1, 265, 0, 0.5266749858856201)
(1, 266, 0, 0.4895622730255127)
(1, 267, 0, 0.5690469741821289)
(1, 268, 0, 0.5576951503753662)
(1, 269, 0, 0.5696868896484375)
(1, 270, 0, 0.5611491203308105)
(1, 271, 0, 0.5804882049560547)
(1, 272, 0, 0.5140228271484375)
(1, 273, 0, 0.5479810237884521)
(1, 274, 0, 0.5766980648040771)
(1, 275, 0, 0.5939819812774658)
(1, 276, 0, 0.5487837791442871)
(1, 277, 0, 0.5560650825500488)
(1, 278, 0, 0.5423669815063477)
(1, 279, 0, 0.6308088302612305)
(1, 280, 0, 0.1799170970916748)
(1, 281, 0, 0.5313270092010498)
(1, 282, 0, 0.5235986709594727)
(1, 283, 0, 0.4826240539550781)
(1, 284, 0, 0.6177141666412354)
(1, 285, 0, 0.5917172431945801)
(1, 286, 0, 0.5376057624816895)
(1, 287, 0, 0.5867969989776611)
(1, 288,

(2, 14, 0, 0.1738729476928711)
(2, 15, 0, 0.02498602867126465)
(2, 16, 0, 0.16725873947143555)
(2, 17, 0, 0.17157888412475586)
(2, 18, 0, 0.033267974853515625)
(2, 19, 0, 0.14638209342956543)
(2, 20, 0, 0.17345428466796875)
(2, 21, 0, 0.16148710250854492)
(2, 22, 0, 0.12293004989624023)
(2, 23, 0, 0.14387774467468262)
(2, 24, 0, 0.17268681526184082)
(2, 25, 0, 0.03193378448486328)
(2, 26, 0, 0.15855073928833008)
(2, 27, 0, 0.12007808685302734)
(2, 28, 0, 0.15225005149841309)
(2, 29, 0, 0.15934014320373535)
(2, 30, 0, 0.17266106605529785)
(2, 31, 0, 0.18255114555358887)
(2, 32, 0, 0.21558403968811035)
(2, 33, 0, 0.21556901931762695)
(2, 34, 0, 0.19715213775634766)
(2, 35, 0, 0.20089316368103027)
(2, 36, 0, 0.2105269432067871)
(2, 37, 0, 0.17075300216674805)
(2, 38, 0, 0.03354692459106445)
(2, 39, 0, 0.20632386207580566)
(2, 40, 0, 0.2140209674835205)
(2, 41, 0, 0.16927719116210938)
(2, 42, 0, 0.21716713905334473)
(2, 43, 0, 0.20796680450439453)
(2, 44, 0, 0.1715831756591797)
(2, 45, 0, 

(2, 269, 0, 0.5832786560058594)
(2, 270, 0, 0.5719070434570312)
(2, 271, 0, 0.6147980690002441)
(2, 272, 0, 0.20238780975341797)
(2, 273, 0, 0.5811851024627686)
(2, 274, 0, 0.5732719898223877)
(2, 275, 0, 0.5927028656005859)
(2, 276, 0, 0.569598913192749)
(2, 277, 0, 0.5855412483215332)
(2, 278, 0, 0.201735258102417)
(2, 279, 0, 0.647942304611206)
(2, 280, 0, 0.5407366752624512)
(2, 281, 0, 0.6366169452667236)
(2, 282, 0, 0.6329951286315918)
(2, 283, 0, 0.6555039882659912)
(2, 284, 0, 0.2051241397857666)
(2, 285, 0, 0.6042580604553223)
(2, 286, 0, 0.6283719539642334)
(2, 287, 0, 0.23456573486328125)
(2, 288, 0, 0.5616717338562012)
(2, 289, 0, 0.5890359878540039)
(2, 290, 0, 0.6299991607666016)
(2, 291, 0, 0.6073801517486572)
(2, 292, 0, 0.598059892654419)
(2, 293, 0, 0.6493570804595947)
(2, 294, 0, 0.6516380310058594)
(2, 295, 0, 0.5961809158325195)
(2, 296, 0, 0.5556879043579102)
(2, 297, 0, 0.5701570510864258)
(2, 298, 0, 0.636786937713623)
(2, 299, 0, 0.6919019222259521)
(2, 300, 0,

(3, 30, 0, 0.20425987243652344)
(3, 31, 0, 0.2091200351715088)
(3, 32, 0, 0.21290802955627441)
(3, 33, 0, 0.2108442783355713)
(3, 34, 0, 0.18918681144714355)
(3, 35, 0, 0.20559906959533691)
(3, 36, 0, 0.17832088470458984)
(3, 37, 0, 0.2204139232635498)
(3, 38, 0, 0.17198896408081055)
(3, 39, 0, 0.1682281494140625)
(3, 40, 0, 0.21024632453918457)
(3, 41, 0, 0.18127012252807617)
(3, 42, 0, 0.19864988327026367)
(3, 43, 0, 0.1902921199798584)
(3, 44, 0, 0.17155885696411133)
(3, 45, 0, 0.18423891067504883)
(3, 46, 0, 0.196854829788208)
(3, 47, 0, 0.19726204872131348)
(3, 48, 0, 0.16011595726013184)
(3, 49, 0, 0.20112299919128418)
(3, 50, 0, 0.21696996688842773)
(3, 51, 0, 0.21439695358276367)
(3, 52, 0, 0.2173619270324707)
(3, 53, 0, 0.03779482841491699)
(3, 54, 0, 0.18464422225952148)
(3, 55, 0, 0.21306920051574707)
(3, 56, 0, 0.21727681159973145)
(3, 57, 0, 0.22115588188171387)
(3, 58, 0, 0.1897108554840088)
(3, 59, 0, 0.21087002754211426)
(3, 60, 0, 0.18418097496032715)
(3, 61, 0, 0.0380

(3, 286, 0, 0.5684638023376465)
(3, 287, 0, 0.605828046798706)
(3, 288, 0, 0.4986739158630371)
(3, 289, 0, 0.48671793937683105)
(3, 290, 0, 0.5859220027923584)
(3, 291, 0, 0.5722870826721191)
(3, 292, 0, 0.5793631076812744)
(3, 293, 0, 0.23615789413452148)
(3, 294, 0, 0.6016080379486084)
(3, 295, 0, 0.6394550800323486)
(3, 296, 0, 0.6094851493835449)
(3, 297, 0, 0.19007611274719238)
(3, 298, 0, 0.6260709762573242)
(3, 299, 0, 0.5473809242248535)
(3, 300, 0, 0.5809628963470459)
(3, 301, 0, 0.6581099033355713)
(3, 302, 0, 0.6313190460205078)
(3, 303, 0, 0.6506171226501465)
(3, 304, 0, 0.5444939136505127)
(3, 305, 0, 0.6101272106170654)
(3, 306, 0, 0.5720489025115967)
(3, 307, 0, 0.6702392101287842)
(3, 308, 0, 0.61313796043396)
(3, 309, 0, 0.6622788906097412)
(3, 310, 0, 0.6608328819274902)
(3, 311, 0, 0.26120495796203613)
(3, 312, 0, 0.6611940860748291)
(3, 313, 0, 0.6875629425048828)
(3, 314, 0, 0.6074302196502686)
(3, 315, 0, 0.678779125213623)
(3, 316, 0, 0.692188024520874)
(3, 317, 

(4, 44, 0, 0.21943235397338867)
(4, 45, 0, 0.16548585891723633)
(4, 46, 0, 0.2075788974761963)
(4, 47, 0, 0.20345282554626465)
(4, 48, 0, 0.15693902969360352)
(4, 49, 0, 0.07503485679626465)
(4, 50, 0, 0.17260098457336426)
(4, 51, 0, 0.18504977226257324)
(4, 52, 0, 0.21493983268737793)
(4, 53, 0, 0.1985621452331543)
(4, 54, 0, 0.03744101524353027)
(4, 55, 0, 0.18622589111328125)
(4, 56, 0, 0.03690338134765625)
(4, 57, 0, 0.22178983688354492)
(4, 58, 0, 0.19446992874145508)
(4, 59, 0, 0.2199258804321289)
(4, 60, 0, 0.2143690586090088)
(4, 61, 0, 0.2163701057434082)
(4, 62, 0, 0.24299192428588867)
(4, 63, 0, 0.24856901168823242)
(4, 64, 0, 0.25443387031555176)
(4, 65, 0, 0.2593259811401367)
(4, 66, 0, 0.24748682975769043)
(4, 67, 0, 0.25158190727233887)
(4, 68, 0, 0.23235511779785156)
(4, 69, 0, 0.2624821662902832)
(4, 70, 0, 0.21701812744140625)
(4, 71, 0, 0.27036380767822266)
(4, 72, 0, 0.26063108444213867)
(4, 73, 0, 0.21656394004821777)
(4, 74, 0, 0.2591419219970703)
(4, 75, 0, 0.257

(4, 299, 0, 0.5921709537506104)
(4, 300, 0, 0.5624189376831055)
(4, 301, 0, 0.6310029029846191)
(4, 302, 0, 0.2171919345855713)
(4, 303, 0, 0.672680139541626)
(4, 304, 0, 0.6001410484313965)
(4, 305, 0, 0.5973680019378662)
(4, 306, 0, 0.5985939502716064)
(4, 307, 0, 0.6479282379150391)
(4, 308, 0, 0.5686028003692627)
(4, 309, 0, 0.6677801609039307)
(4, 310, 0, 0.6456000804901123)
(4, 311, 0, 0.6858549118041992)
(4, 312, 0, 0.5661990642547607)
(4, 313, 0, 0.6856510639190674)
(4, 314, 0, 0.6411159038543701)
(4, 315, 0, 0.5604610443115234)
(4, 316, 0, 0.6297857761383057)
(4, 317, 0, 0.7139298915863037)
(4, 318, 0, 0.7191579341888428)
(4, 319, 0, 0.7435088157653809)
(4, 320, 0, 0.6276712417602539)
(4, 321, 0, 0.6227149963378906)
(4, 322, 0, 0.6393449306488037)
(4, 323, 0, 0.6602938175201416)
(4, 324, 0, 0.5605230331420898)
(4, 325, 0, 0.611259937286377)
(4, 326, 0, 0.660146951675415)
(4, 327, 0, 0.6378817558288574)
(4, 328, 0, 0.5711350440979004)
(4, 329, 0, 0.6853201389312744)
(4, 330, 0,

(5, 59, 0, 0.18470406532287598)
(5, 60, 0, 0.04156494140625)
(5, 61, 0, 0.2196662425994873)
(5, 62, 0, 0.25701284408569336)
(5, 63, 0, 0.22490715980529785)
(5, 64, 0, 0.2666299343109131)
(5, 65, 0, 0.2317371368408203)
(5, 66, 0, 0.2672607898712158)
(5, 67, 0, 0.2500128746032715)
(5, 68, 0, 0.23436903953552246)
(5, 69, 0, 0.26062893867492676)
(5, 70, 0, 0.23059678077697754)
(5, 71, 0, 0.25847411155700684)
(5, 72, 0, 0.22549676895141602)
(5, 73, 0, 0.25050878524780273)
(5, 74, 0, 0.2465968132019043)
(5, 75, 0, 0.25251197814941406)
(5, 76, 0, 0.20131707191467285)
(5, 77, 0, 0.23825430870056152)
(5, 78, 0, 0.04760909080505371)
(5, 79, 0, 0.24633407592773438)
(5, 80, 0, 0.26990699768066406)
(5, 81, 0, 0.27115893363952637)
(5, 82, 0, 0.22298789024353027)
(5, 83, 0, 0.27410101890563965)
(5, 84, 0, 0.09629011154174805)
(5, 85, 0, 0.26020097732543945)
(5, 86, 0, 0.2768850326538086)
(5, 87, 0, 0.27396392822265625)
(5, 88, 0, 0.052932024002075195)
(5, 89, 0, 0.2531771659851074)
(5, 90, 0, 0.25550

(5, 314, 0, 0.6512749195098877)
(5, 315, 0, 0.24996495246887207)
(5, 316, 0, 0.6943559646606445)
(5, 317, 0, 0.2595338821411133)
(5, 318, 0, 0.6011531352996826)
(5, 319, 0, 0.7465238571166992)
(5, 320, 0, 0.5986049175262451)
(5, 321, 0, 0.6445319652557373)
(5, 322, 0, 0.550105094909668)
(5, 323, 0, 0.6837420463562012)
(5, 324, 0, 0.11512494087219238)
(5, 325, 0, 0.6224820613861084)
(5, 326, 0, 0.1265239715576172)
(5, 327, 0, 0.6937830448150635)
(5, 328, 0, 0.6574809551239014)
(5, 329, 0, 0.6133260726928711)
(5, 330, 0, 0.6651129722595215)
(5, 331, 0, 0.6497349739074707)
(5, 332, 0, 0.6588881015777588)
(5, 333, 0, 0.6338338851928711)
(5, 334, 0, 0.6746840476989746)
(5, 335, 0, 0.622941255569458)
(5, 336, 0, 0.5979161262512207)
(5, 337, 0, 0.6967501640319824)
(5, 338, 0, 0.6770980358123779)
(5, 339, 0, 0.6521291732788086)
(5, 340, 0, 0.635887861251831)
(5, 341, 0, 0.7268369197845459)
(5, 342, 0, 0.6631848812103271)
(5, 343, 0, 0.7699949741363525)
(5, 344, 0, 0.6588377952575684)
(5, 345, 

In [3]:
# verify narrow version
import time

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        d=order(2,a)
        purepart=2^d
        oddpart=a/2^d
        #ord(2,tau(2^n))= (I believe) 2d
        # so ord(2,tau(purepart))=ord(2,tau(2^d)) = 2n
        # so tau(purepart) = 2^(2d)
        # so tau(a) = tau(purepart)*tau(oddpart)=
        # 4^d*tau(oddpart); then we replace tau(oddpart)
        # with sigma(oddpart,k=11) by Ramanujan's congruence
        term=4^d*sigma(oddpart,k=11)
        ans=ans+term*x^a
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

ls=[]
start=time.time()
for a in [1..2000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    o2=order(2,c)
    ds=digitsum(2,a)
    test=o2-3*ds+order(2,a+1) # 3rd term based on some observations
    # this will verify the 'rules' for even a.
    finish=time.time()
    print((a,test,finish-start))
    if test!=0:ls=ls+[a]
print()
print("no-count:");print();print(len(s))
print("bad a's:");print();print(ls)

(1, 0, 0.005185127258300781)
(2, 0, 0.026188135147094727)
(3, 0, 0.05177593231201172)
(4, 0, 0.08165407180786133)
(5, 0, 0.11594986915588379)
(6, 0, 0.17346787452697754)
(7, 0, 0.2405540943145752)
(8, 0, 0.29813289642333984)
(9, 0, 0.3552539348602295)
(10, 0, 0.412369966506958)
(11, 0, 0.469465970993042)
(12, 0, 0.5311579704284668)
(13, 0, 0.5892727375030518)
(14, 0, 0.6485819816589355)
(15, 0, 0.7070579528808594)
(16, 0, 0.7652370929718018)
(17, 0, 0.829010009765625)
(18, 0, 0.8875401020050049)
(19, 0, 0.9496347904205322)
(20, 0, 1.0133130550384521)
(21, 0, 1.0846459865570068)
(22, 0, 1.1494379043579102)
(23, 0, 1.2141239643096924)
(24, 0, 1.2829258441925049)
(25, 0, 1.3496551513671875)
(26, 0, 1.415092945098877)
(27, 0, 1.4802489280700684)
(28, 0, 1.5450260639190674)
(29, 0, 1.6092028617858887)
(30, 0, 1.674133062362671)
(31, 0, 1.7404799461364746)
(32, 0, 1.8107609748840332)
(33, 0, 1.8777759075164795)
(34, 0, 1.9453229904174805)
(35, 0, 2.011125087738037)
(36, 0, 2.0763099193573)
(

(293, 0, 24.82690405845642)
(294, 0, 24.950381755828857)
(295, 0, 25.0697820186615)
(296, 0, 25.18810796737671)
(297, 0, 25.308468103408813)
(298, 0, 25.428517818450928)
(299, 0, 25.553837060928345)
(300, 0, 25.674668073654175)
(301, 0, 25.80098795890808)
(302, 0, 25.926846981048584)
(303, 0, 26.05464196205139)
(304, 0, 26.174019813537598)
(305, 0, 26.29595184326172)
(306, 0, 26.422677040100098)
(307, 0, 26.550153017044067)
(308, 0, 26.675480127334595)
(309, 0, 26.8035409450531)
(310, 0, 26.935920000076294)
(311, 0, 27.068145990371704)
(312, 0, 27.192461013793945)
(313, 0, 27.323330879211426)
(314, 0, 27.45178198814392)
(315, 0, 27.58246612548828)
(316, 0, 27.71477198600769)
(317, 0, 27.84931492805481)
(318, 0, 27.9826979637146)
(319, 0, 28.119823932647705)
(320, 0, 28.24175500869751)
(321, 0, 28.368791103363037)
(322, 0, 28.494508028030396)
(323, 0, 28.625128984451294)
(324, 0, 28.750678062438965)
(325, 0, 28.88243794441223)
(326, 0, 29.011962890625)
(327, 0, 29.144803762435913)
(328,

(585, 0, 85.79381084442139)
(586, 0, 86.10007882118225)
(587, 0, 86.43108797073364)
(588, 0, 86.73904800415039)
(589, 0, 87.06851410865784)
(590, 0, 87.39836096763611)
(591, 0, 87.74951791763306)
(592, 0, 88.05500292778015)
(593, 0, 88.38238000869751)
(594, 0, 88.70897316932678)
(595, 0, 89.05512189865112)
(596, 0, 89.39694499969482)
(597, 0, 89.75012993812561)
(598, 0, 90.09820914268494)
(599, 0, 90.46444606781006)
(600, 0, 90.78929376602173)
(601, 0, 91.13845086097717)
(602, 0, 91.4797739982605)
(603, 0, 91.85243701934814)
(604, 0, 92.20065379142761)
(605, 0, 92.56979990005493)
(606, 0, 92.94546580314636)
(607, 0, 93.34254693984985)
(608, 0, 93.65708708763123)
(609, 0, 93.99243092536926)
(610, 0, 94.3268358707428)
(611, 0, 94.67136096954346)
(612, 0, 94.99568390846252)
(613, 0, 95.34845185279846)
(614, 0, 95.70931577682495)
(615, 0, 96.08303594589233)
(616, 0, 96.41262912750244)
(617, 0, 96.76514887809753)
(618, 0, 97.11050081253052)
(619, 0, 97.48729085922241)
(620, 0, 97.8483970165

(872, 0, 216.28031492233276)
(873, 0, 216.88840293884277)
(874, 0, 217.48983883857727)
(875, 0, 218.1369128227234)
(876, 0, 218.74099898338318)
(877, 0, 219.3859179019928)
(878, 0, 220.03596806526184)
(879, 0, 220.7358889579773)
(880, 0, 221.33883810043335)
(881, 0, 221.97342109680176)
(882, 0, 222.613214969635)
(883, 0, 223.3607177734375)
(884, 0, 224.06729817390442)
(885, 0, 224.78383898735046)
(886, 0, 225.4844307899475)
(887, 0, 226.2133378982544)
(888, 0, 226.85817098617554)
(889, 0, 227.54926705360413)
(890, 0, 228.2381718158722)
(891, 0, 229.01900911331177)
(892, 0, 229.74973607063293)
(893, 0, 230.50251984596252)
(894, 0, 231.27477502822876)
(895, 0, 232.0746328830719)
(896, 0, 232.62655591964722)
(897, 0, 233.20323586463928)
(898, 0, 233.78837776184082)
(899, 0, 234.4109308719635)
(900, 0, 235.0005030632019)
(901, 0, 235.63592886924744)
(902, 0, 236.26761293411255)
(903, 0, 236.93125414848328)
(904, 0, 237.5201280117035)
(905, 0, 238.15145301818848)
(906, 0, 238.77321815490723

(1155, 0, 462.2913830280304)
(1156, 0, 463.26873087882996)
(1157, 0, 464.3194739818573)
(1158, 0, 465.3690209388733)
(1159, 0, 466.48853302001953)
(1160, 0, 467.46401476860046)
(1161, 0, 468.5130398273468)
(1162, 0, 469.56148982048035)
(1163, 0, 470.6672341823578)
(1164, 0, 471.713339805603)
(1165, 0, 472.83514404296875)
(1166, 0, 473.9469430446625)
(1167, 0, 475.16525292396545)
(1168, 0, 476.1641700267792)
(1169, 0, 477.2465829849243)
(1170, 0, 478.3212559223175)
(1171, 0, 479.47389006614685)
(1172, 0, 480.5937280654907)
(1173, 0, 481.7490699291229)
(1174, 0, 482.90545892715454)
(1175, 0, 484.1545159816742)
(1176, 0, 485.2400047779083)
(1177, 0, 486.4094500541687)
(1178, 0, 487.56491899490356)
(1179, 0, 488.7955799102783)
(1180, 0, 489.9536221027374)
(1181, 0, 491.20517897605896)
(1182, 0, 492.4482808113098)
(1183, 0, 493.78037691116333)
(1184, 0, 494.79097604751587)
(1185, 0, 495.8799729347229)
(1186, 0, 496.97436904907227)
(1187, 0, 498.15915608406067)
(1188, 0, 499.2940628528595)
(

(1438, 0, 860.8460669517517)
(1439, 0, 862.8180618286133)
(1440, 0, 864.3388979434967)
(1441, 0, 865.9895141124725)
(1442, 0, 867.6585969924927)
(1443, 0, 869.4184019565582)
(1444, 0, 871.0617759227753)
(1445, 0, 872.8131358623505)
(1446, 0, 874.5982117652893)
(1447, 0, 876.4749729633331)
(1448, 0, 878.1174919605255)
(1449, 0, 879.8781960010529)
(1450, 0, 881.6689119338989)
(1451, 0, 883.5542190074921)
(1452, 0, 885.3178279399872)
(1453, 0, 887.2035238742828)
(1454, 0, 889.1168761253357)
(1455, 0, 891.1218068599701)
(1456, 0, 892.772852897644)
(1457, 0, 894.5411458015442)
(1458, 0, 896.3280410766602)
(1459, 0, 898.2235400676727)
(1460, 0, 899.9954698085785)
(1461, 0, 901.8936009407043)
(1462, 0, 903.7916197776794)
(1463, 0, 905.8032240867615)
(1464, 0, 907.5716559886932)
(1465, 0, 909.455974817276)
(1466, 0, 911.3851850032806)
(1467, 0, 913.4185919761658)
(1468, 0, 915.3202729225159)
(1469, 0, 917.3309698104858)
(1470, 0, 919.3786640167236)
(1471, 0, 921.5565991401672)
(1472, 0, 923.14

(1716, 0, 1455.6621057987213)
(1717, 0, 1458.34095287323)
(1718, 0, 1461.0138499736786)
(1719, 0, 1463.8474729061127)
(1720, 0, 1466.367031097412)
(1721, 0, 1469.05526304245)
(1722, 0, 1471.7488219738007)
(1723, 0, 1474.6061718463898)
(1724, 0, 1477.317745923996)
(1725, 0, 1480.1930119991302)
(1726, 0, 1483.0819590091705)
(1727, 0, 1486.184865951538)
(1728, 0, 1488.428375005722)
(1729, 0, 1490.8478419780731)
(1730, 0, 1493.260437965393)
(1731, 0, 1495.8095469474792)
(1732, 0, 1498.1911709308624)
(1733, 0, 1500.729101896286)
(1734, 0, 1503.4529399871826)
(1735, 0, 1506.1729249954224)
(1736, 0, 1508.5589880943298)
(1737, 0, 1511.10293507576)
(1738, 0, 1513.6546449661255)
(1739, 0, 1516.394581079483)
(1740, 0, 1518.9723258018494)
(1741, 0, 1521.7044081687927)
(1742, 0, 1524.4536111354828)
(1743, 0, 1527.3711388111115)
(1744, 0, 1529.7776288986206)
(1745, 0, 1532.3734538555145)
(1746, 0, 1534.9543409347534)
(1747, 0, 1537.7188720703125)
(1748, 0, 1540.3075819015503)
(1749, 0, 1543.04737305

(1993, 0, 2287.6226019859314)
(1994, 0, 2291.0059390068054)
(1995, 0, 2294.5886929035187)
(1996, 0, 2297.9769990444183)
(1997, 0, 2301.6221330165863)
(1998, 0, 2305.255394935608)
(1999, 0, 2309.0304980278015)
(2000, 0, 2312.185231924057)

no-count:



NameError: name 's' is not defined